In [1]:
pip install refl1d


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install seaborn

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install plotly

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install ipympl

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import sys
import os
import math
import json
import numpy as np

import refl1d
from refl1d.names import *

#import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
%matplotlib inline
import seaborn as sea

In [5]:
#This block is the class that is used to generate the SLD and NR curves

from scipy.interpolate import CubicSpline
import random
def calculate_reflectivity(q, model_description, q_resolution=0.0294855):
    """
        Reflectivity calculation using refl1d
    """
    # print(model_description['scale'], model_description['background'])
    zeros = np.zeros(len(q))
    dq = q_resolution * q / 2.355

    # The QProbe object represents the beam
    probe = QProbe(q, dq, data=(zeros, zeros))

    layers = model_description['layers']
    sample = Slab(material=SLD(name=layers[0]['name'],
                               rho=layers[0]['sld']), interface=layers[0]['roughness'])
    # Add each layer
    for l in layers[1:]:
        sample = sample | Slab(material=SLD(name=l['name'],
                               rho=l['sld'], irho=l['isld']),
                               thickness=l['thickness'], interface=l['roughness'])

    probe.background = Parameter(value=model_description['background'], name='background')
    expt = Experiment(probe=probe, sample=sample)
    # print(model_description)
    q, r = expt.reflectivity()
    return model_description['scale'] * r
class ReflectivityModels(object):
        # Neutrons come in from the last item in the list

    def __init__(self,q=None, name='polymer'):

        self.parameters = [
                              dict(i=0, par='roughness', bounds=[1,3]),

                  # The following is the Si oxide layer
                  dict(i=1, par='thickness', bounds=[10,15]),
                  dict(i=1, par='roughness', bounds=[1,3]),

                  # The next two layers are the polymer
                    dict(i=2, par='sld', bounds=[0,6]),
                  dict(i=2, par='thickness', bounds=[0,200]),
                  dict(i=2, par='roughness', bounds=[0,150]),
                    dict(i=3, par='sld', bounds=[0,6]),
                  dict(i=3, par='thickness', bounds=[0,200]),
                  dict(i=3, par='roughness', bounds=[0,150]),
                    dict(i=4, par='sld', bounds=[0,6]),
                  dict(i=4, par='thickness', bounds=[0,200]),
                  dict(i=4, par='roughness', bounds=[0,150]),
                  dict(i=5, par='sld', bounds=[0,6]),
                  dict(i=5, par='thickness', bounds=[0,200]),
                  dict(i=5, par='roughness', bounds=[0,150]),
                  dict(i=6, par='sld', bounds=[0,6]),
                  dict(i=6, par='thickness', bounds=[0,200]),
                  dict(i=6, par='roughness', bounds=[0,150]),
            # dict(i=7, par='sld', bounds=[0,6]),
            #       dict(i=7, par='thickness', bounds=[0,200]),
            #       dict(i=7, par='roughness', bounds=[0,150]),
            # dict(i=8, par='sld', bounds=[0,6]),
            #       dict(i=8, par='thickness', bounds=[0,200]),
            #       dict(i=8, par='roughness', bounds=[0,150]),
            # dict(i=9, par='sld', bounds=[0,6]),
            #       dict(i=9, par='thickness', bounds=[0,200]),
            #       dict(i=9, par='roughness', bounds=[0,150]),
            # dict(i=10, par='sld', bounds=[0,6]),
            #       dict(i=10, par='thickness', bounds=[0,200]),
            #       dict(i=10, par='roughness', bounds=[0,150]),
            # dict(i=10, par='sld', bounds=[0,6]),
            #       dict(i=10, par='thickness', bounds=[0,200]),
            #       dict(i=10, par='roughness', bounds=[0,150]),
                 ]
        self.model_description = dict(layers=[
                                dict(sld=2.07, isld=0, thickness=0, roughness=1.8272, name='substrate'),
                                dict(sld=3.47, isld=0, thickness=10.085, roughness=1.108, name='siox'),
                                dict(sld=3.734, isld=0, thickness=8.717, roughness=2.2571, name='bottom'),
                                dict(sld=3.1033, isld=0, thickness=239.91, roughness=37.707, name='bottom'),
                                dict(sld=3.0246, isld=0, thickness=91.232, roughness=20.147, name='bulk'),
                                dict(sld=3.0246, isld=0, thickness=51.655, roughness=20.147, name='bulk2'),
                                dict(sld=0.85516, isld=0, thickness=62.189, roughness=43.155, name='bottom'),
            # dict(sld=0.85516, isld=0, thickness=62.189, roughness=43.155, name='bottom1'),
            # dict(sld=0.85516, isld=0, thickness=62.189, roughness=43.155, name='bottom2'),
            # dict(sld=0.85516, isld=0, thickness=62.189, roughness=43.155, name='bottom3'),
            # dict(sld=0.85516, isld=0, thickness=62.189, roughness=43.155, name='bottom4'),
                                dict(sld=0, isld=0, thickness=0, roughness=0, name='air')
                         ],
                         scale=[0.85,1],
                         background=[0.000001,0.000004]
                        )
        # The following are unmodified physical parameters and corresponding reflectivity data
        self._pars_array = []
        self._refl_array = []
        self._edited_pars_array = []
        # This creates the SLD curve list
        # The following are the parameters, mapped between -1 and 1.
        self._train_pars = []
        self._train_data = None
        self._config_name = name

        if q is None:
            self.q = np.logspace(np.log10(0.008101436040354381), np.log10(0.1975709062238298), num=308)
        else:
            self.q = q

    @classmethod
    def from_dict(cls, pars, q_array=None):
        """
            Create ReflectivityModels object from a dict that
            defines the reflectivity model parameters and how
            the training set should be generated.
        """
        m = cls(q_array, name=pars['name'])
        m.model_description =  pars['model']
        m.parameters = pars['parameters']
        return m

    def generate(self, n=100):
        """
            Generate a random sample of models
        """
        npars = len(self.parameters)
        self._train_pars = np.random.uniform(low=-1, high=1, size=[n, npars])
        self._train_scale = np.random.uniform(low=-1, high=1, size=[n])
        self._train_bkg = np.random.uniform(low=-1, high=1, size=[n])
        # Compute model parameters and reflectivity using these values
        self.compute_reflectivity()

    def to_model_parameters(self, pars, scale, bkg):
        """
            Transform an array of parameters to a list of calculable models
        """
        pars_array = np.zeros(pars.shape)
        scale_array = np.zeros(scale.shape)
        bkg_array = np.zeros(bkg.shape)
        # print(self.parameters) 
        for i, par in enumerate(self.parameters):
            a = (par['bounds'][1]-par['bounds'][0])/2.
            b = (par['bounds'][1]+par['bounds'][0])/2.
            pars_array.T[i] = pars.T[i] * a + b
        c = (self.model_description['scale'][1]-self.model_description['scale'][0])/2.
        d = (self.model_description['scale'][1]+self.model_description['scale'][0])/2.
        scale_array = scale * c + d
        e = (self.model_description['background'][1]-self.model_description['background'][0])/2.
        f = (self.model_description['background'][1]+self.model_description['background'][0])/2.
        bkg_array = bkg * e + f
        # print(pars)
        return pars_array, scale_array, bkg_array


    def compute_reflectivity(self):
        """
            Transform an array of parameters to a list of calculable models
            and compute reflectivity
        """
        print("Computing reflectivity")
        self._pars_array, self._scale_array, self._bkg_array = self.to_model_parameters(self._train_pars, self._train_scale, self._train_bkg)
        for i, p in enumerate(self._pars_array):
            # print(p)
            _desc = self.get_model_description(p, self._scale_array[i], self._bkg_array[i])
            r = calculate_reflectivity(self.q, _desc)
            self._edited_pars_array.append(np.concatenate((p,[self._scale_array[i], self._bkg_array[i]])))
            self._refl_array.append(r)
    def get_model_description(self, pars, scale, bkg):
        """
            Convert the parameter list to a model description that we can use
            to compute R(q).
        """
        # print(self.parameters)
        for i, par in enumerate(self.parameters):
            self.model_description['layers'][par['i']][par['par']] = pars[i]
        self.model_description['scale'] = scale
        self.model_description['background'] = bkg
        return self.model_description

    def get_preprocessed_data(self, errors=None):
        """
            Pre-process data
            If errors is provided, a random error will be added, taking the errors array
            as a relative uncertainty.
        """
        if errors is None:

            self._train_data = np.log10(self._refl_array*self.q**2/self.q[0]**2)
        else:
            _data = self._refl_array * (1.0 + np.random.normal(size=len(errors)) * errors)
            # Catch the few cases where we generated a negative intensity and take
            # the absolute value
            _data[_data<0] = np.fabs(_data[_data<0])
            self._train_data = np.log10(_data*self.q**2/self.q[0]**2)

        return self._train_pars, self._train_data

    def save(self, output_dir=''):
        """
            Save all data relevant to a training set
            @param output_dir: directory used to store training sets
        """
        # Save q values
        np.save(os.path.join(output_dir, "%s_q_values" % self._config_name), self.q)

        # Save training set
        if self._train_data is not None:
            np.save(os.path.join(output_dir, "%s_data" % self._config_name), self._train_data)
            np.save(os.path.join(output_dir, "%s_pars" % self._config_name), self._train_pars)

    def load(self, data_dir=''):
        self.q = np.load(os.path.join(data_dir, "%s_q_values.npy" % self._config_name))
        self._train_data = np.load(os.path.join(data_dir, "%s_data.npy" % self._config_name))
        self._train_pars = np.load(os.path.join(data_dir, "%s_pars.npy" % self._config_name))
        return self.q, self._train_data, self._train_pars


In [6]:
m = ReflectivityModels()
m.generate(3000000)
pars, train_data = m.get_preprocessed_data()
# print(m._refl_array.shape)
settingUp = []
SLDSet = []
for i in range(len(m._refl_array)):
    settingUp.append(np.array([m.q,m._refl_array[i]]))
# plt.plot(m.q, m._refl_array[0])
# plt.xscale('log')
# plt.yscale('log')
totalStack = np.stack(settingUp)
# print(parameters.shape)
parameters = np.stack(m._edited_pars_array)

np.save("ParametersSpecialFull.npy", parameters)
np.save("NRSpecialFull.npy", totalStack)

Computing reflectivity
